# Services (daily)

In [19]:
import pandas as pd

## Database connection

In [20]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [21]:
# Load dimensions
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table("SERVICE_STATUS_DIMENSION", OLAP_connection)

In [22]:
# Load fact table data from OLTP
services = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_statuses = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)
users_information = pd.read_sql_table("clientes_usuarioaquitoy", OLTP_connection)

## Transformation

In [23]:
services = services.dropna(subset=["mensajero_id"])

In [24]:
services = services[[
		"id",
		"fecha_solicitud",
		"cliente_id",
		"mensajero_id",
		"usuario_id"
	]]

In [25]:
latest_statuses = service_statuses.sort_values("fecha").drop_duplicates(
		"servicio_id", keep="last"
)

In [26]:
latest_statuses = latest_statuses[["estado_id", "servicio_id"]]

In [27]:
latest_statuses = latest_statuses.rename(
		columns={
				"id": "status_record_id",
				"estado_id": "status_id",
				"servicio_id": "service_id"
		}
)

In [28]:
services = services.rename(
		columns={
				"id": "service_id",
				"fecha_solicitud": "date",
				"cliente_id": "original_customer_id",
				"mensajero_id": "original_courier_id"
		}
)

In [29]:
services = services.merge(
		latest_statuses, on="service_id", how="left"
	)

In [30]:
services = services.merge(users_information[["id", "sede_id"]], left_on="usuario_id", right_on="id",
							how="inner")
services.drop(columns=["usuario_id", "id"], inplace=True)

In [31]:
services = services.merge(office_dimension[["office_id", "original_office_id"]], left_on="sede_id",
							right_on="original_office_id", how="inner")
services.drop(columns=["sede_id", "original_office_id"], inplace=True)

In [32]:
services = pd.merge(services, time_dimension[["time_id", "date"]], on="date", how="inner")
services.drop(columns=["date", "status_id"], inplace=True)

In [33]:
services = (services.groupby(["original_customer_id", "original_courier_id", "time_id", "office_id"])
				.size().reset_index(name="total_services"))

In [34]:
service_fact = services.merge(customer_dimension[["customer_id", "original_customer_id"]], on="original_customer_id", how="inner")
service_fact.drop(columns=["original_customer_id"], inplace=True)

In [ ]:
service_fact = service_fact.merge(courier_dimension[["courier_id", "original_courier_id"]], on="original_courier_id",
								how="inner")
service_fact.drop(columns=["original_courier_id"], inplace=True)

In [21]:
service_fact.reset_index(inplace=True)
service_fact = service_fact.rename(columns={ "index": "service_daily_id" })
service_fact.set_index("service_daily_id", inplace=True)

## Load

In [22]:
from sqlalchemy import BigInteger


dtype_mapping = {
	"time_id": BigInteger(),
	"customer_id": BigInteger(),
	"courier_id": BigInteger(),
	"office_id": BigInteger(),
}

In [23]:
# Save the fact table to OLAP
service_fact.to_sql(
	"SERVICE_DAILY_FACT_TABLE", OLAP_connection, if_exists="replace", index=True, dtype=dtype_mapping
)

535